# Python on-the-fly colormap generator

Useful bit of code that allows you to see how python plots look on-the-fly.

In [1]:
import Make_EQ as eq
from ipywidgets import interactive, VBox, Layout, widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import os

In [2]:
# define output interpolation specifics
VMIN=272.15
VMAX=308.16
VSTEPS=255

In [3]:
%matplotlib inline

# MEDSPIRATION
#channel_red=[1,1,0,0,0,0,0,0.5,1,1,1,0.25]
#channel_green=[1,0.4,0,0.5,1,0.7,0.42,0.7,1,0.5,0,0]
#channel_blue=[1,1,0.2,0.6,1,0.5,0,0,0,0,0,0]

# ALT. MEDSPIRATION
#channel_red=[1,1,0,0,0,0,0.33,0.66,1,1,1,0.25]
#channel_green=[1,0.4,0,0.5,1,0.6,0.2,0.6,1,0.5,0,0]
#channel_blue=[1,1,0.2,0.6,1,0.5,0,0,0,0,0,0]

# Linear BMY(OR)
channel_red=[0.00,0.11,0.35,0.57,0.75,0.89,1.00,1.00,1.00,1.00,1.00,1.00]
channel_green=[0.06,0.10,0.10,0.05,0.05,0.17,0.35,0.53,0.67,0.81,0.65,0.00]
channel_blue=[0.36,0.51,0.57,0.56,0.52,0.46,0.36,0.28,0.22,0.23,0.00,0.00]

# ALT Linear BMY(OR)
channel_red=[0.00,0.00,0.00,0.1,0.3,0.65,1.00,1.00,1.00,1.00,0.25,0.50]
channel_green=[0.00,0.35,1.00,0.33,0.00,0.00,0.50,1.00,0.28,0.00,0.00,0.50]
channel_blue=[0.25,0.38,1.00,0.72,0.30,0.65,1.00,0.00,0.00,0.00,0.00,0.50]

# test 
channel_red=   [1.00, 0.75, 0.00, 0.00, 0.00, 0.00, 0.10, 0.25, 0.50, 0.75, 1.00, 1.00, 1.00, 0.20, 1.00, 1.00]
channel_green= [1.00, 0.90, 0.10, 0.10, 0.40, 1.00, 0.33, 0.00, 0.25, 0.50, 1.00, 0.65, 0.00, 0.00, 0.65, 0.95]
channel_blue=  [1.00, 1.00, 0.20, 0.20, 0.40, 1.00, 0.60, 0.25, 0.50, 0.75, 0.00, 0.00, 0.00, 0.00, 0.50, 0.95]

print('nChannels: '+str(len(channel_red)))

# build the interactive plotter widgets
red_widgets = [] ; green_widgets = [] ; blue_widgets = []

# build command; not a great way of doing this, but allows for dynamic function construction.
for ii in range(len(channel_red)):
    red_widgets.append('red' + str(ii) + '=' + 'widgets.FloatSlider(value=' \
                       + str(channel_red[ii]) + ', min=0.0, max=1.0, step=0.025)')
    green_widgets.append('green' + str(ii) + '=' + 'widgets.FloatSlider(value=' \
                         + str(channel_green[ii]) + ', min=0.0, max=1.0, step=0.025)')
    blue_widgets.append('blue' + str(ii) + '=' + 'widgets.FloatSlider(value=' \
                        + str(channel_blue[ii]) + ', min=0.0, max=1.0, step=0.025)')

runCMD = 'interactive_plot = interactive(eq.plot_eq' + ',vmin=' + str(VMIN) + ',vmax=' + str(VMAX) + ',' \
                                     +','.join(red_widgets) + ',' \
                                     +','.join(green_widgets) + ',' \
                                     +','.join(blue_widgets) \
                                     +')'

exec(runCMD)
box_layout = Layout(display='flex', flex_flow='row', justify_content='space-between', align_items='center')
for ii in range(2,len(channel_red)+2):
    display(VBox([interactive_plot.children[ii],interactive_plot.children[ii+len(channel_red)],interactive_plot.children[ii+len(channel_red)*2]],\
                 layout=box_layout))
display(interactive_plot.children[-1])

nChannels: 16


Output()

In [4]:
# interpolate colour map onto required range / steps
REDS = interactive_plot.result[2] ; GREENS = interactive_plot.result[3] ; BLUES = interactive_plot.result[4]
TSCALE_ORIG = np.linspace(VMIN, VMAX, len(REDS))
TSCALE_NEW = np.linspace(VMIN, VMAX, VSTEPS)

interp_redf = interpolate.interp1d(TSCALE_ORIG, REDS)
interp_greenf = interpolate.interp1d(TSCALE_ORIG, GREENS)
interp_bluef = interpolate.interp1d(TSCALE_ORIG, BLUES)

interp_red = np.array(interp_redf(TSCALE_NEW)*255.).astype(int)
interp_green = np.array(interp_greenf(TSCALE_NEW)*255.).astype(int)
interp_blue = np.array(interp_bluef(TSCALE_NEW)*255.).astype(int)

In [5]:
# write to file
my_file = 'BL_Test_Map3.grass'
if os.path.exists(my_file):
    os.remove(my_file)
    
with open(my_file, 'w') as the_file:
    for ii in range(0,len(interp_red)):
        the_file.write(str(TSCALE_NEW[ii]) + ' ' + \
                       str(interp_red[ii]) + ' ' + \
                       str(interp_green[ii]) + ' ' + \
                       str(interp_blue[ii]))
        if ii != len(interp_red)-1:
            the_file.write('\n')